In [ ]:
######################################

# ETL Proccess for SECOP TOPICS Project

# Please see file requirements.txt 
# to install the requred libraries

######################################

In [17]:
import sys
import pandas as pd
import numpy as np
from sodapy import Socrata
import itertools
import psycopg2
from psycopg2 import Error


In [18]:
## Conexion to Datos Abiertos de Colombia

client = Socrata("www.datos.gov.co", None)
secopI='xvdy-vvsk'
secopII='jbjy-vk9h' 
count_SECOPII = client.get(secopII,select="COUNT(*)")
count_SECOPI = client.get(secopI,select="COUNT(*)")

In [3]:
## Conexion to AWS Postgres
conn_string="dbname='' port='5432' user='postgres' password='mydb1234' host='ds4adatabase.cfuz9qvi996n.us-east-2.rds.amazonaws.com'"
conn = None
conn=psycopg2.connect(conn_string)
cursor=conn.cursor()
cursor.execute('select version()')
records = cursor.fetchall()
conn.commit()
cursor.close()
conn.close
print(records)

[('PostgreSQL 12.4 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 4.8.3 20140911 (Red Hat 4.8.3-9), 64-bit',)]


In [4]:
## Creating Table to integrated SECOPI & SECOP II
from psycopg2 import Error
try:
    ##Connection
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    
    ##Create table
        
    create_table_query = '''CREATE TABLE IF NOT EXISTS secop (
        NOMBRE_ENTIDAD text,
        NIT_ENTIDAD text,
        DEPARTAMENTO text,
        CIUDAD text,
        ORDEN text,
        ID_CONTRATO text,
        ESTADO_CONTRATO text,
        CODIGO_DE_CATEGORIA_PRINCIPAL text,
        DESCRIPCION_DEL_PROCESO text,
        TIPO_DE_CONTRATO text,
        MODALIDAD_DE_CONTRATACION text,
        FECHA_DE_FIRMA text,
        FECHA_DE_INICIO_DE_EJECUCION text,
        FECHA_DE_FIN_DE_EJECUCION text,
        TIPODOCPROVEEDOR text,
        DOCUMENTO_PROVEEDOR text,
        PROVEEDOR_ADJUDICADO text,
        VALOR_DEL_CONTRATO numeric,
        ORIGEN_DE_LOS_RECURSOS text,
        DIAS_ADICIONADOS bigint,
        FUENTE text
    );'''
    cursor.execute(create_table_query)
    conn.commit()
    print("Table created successfully in PostgreSQL ")
except (Exception, psycopg2.DatabaseError) as error :
    print ("Error while creating PostgreSQL table", error)
finally:
    #closing database connection.
        if(conn):
            cursor.close()
            conn.close()
            print("PostgreSQL connection is closed")

Table created successfully in PostgreSQL 
PostgreSQL connection is closed


In [ ]:
## Inserting information of SECOP II

%%time
insert_table_query = '''INSERT INTO secop(
    nombre_entidad, nit_entidad, departamento, ciudad, 
    orden, id_contrato, estado_contrato, codigo_de_categoria_principal, 
    descripcion_del_proceso, tipo_de_contrato, modalidad_de_contratacion, 
    fecha_de_firma, fecha_de_inicio_de_ejecucion, fecha_de_fin_de_ejecucion, 
    tipodocproveedor, documento_proveedor, proveedor_adjudicado, 
    valor_del_contrato, origen_de_los_recursos, dias_adicionados, fuente)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
start = 0            # Start at 0
chunk_size = 10000    # Fetch 2000 rows at a time
results =[]          # Empty out our result list
while True:
    
     # Fetch the set of records starting at 'start'
    results = client.get(secopII, offset=start, limit=chunk_size,select='''nombre_entidad, nit_entidad,
                                        departamento,ciudad,orden,
                                        id_contrato,estado_contrato,
                                        codigo_de_categoria_principal,descripcion_del_proceso,
                                        tipo_de_contrato,modalidad_de_contratacion,
                                        fecha_de_firma, fecha_de_inicio_de_ejecucion,
                                        fecha_de_fin_de_ejecucion,tipodocproveedor,
                                        documento_proveedor,proveedor_adjudicado,
                                        valor_del_contrato,origen_de_los_recursos,
                                        dias_adicionados,"SECOPII"
                                        ''')

    results = pd.DataFrame(results)
    results = results[['nombre_entidad', 'nit_entidad', 'departamento', 'ciudad', 'orden',
       'id_contrato', 'estado_contrato', 'codigo_de_categoria_principal',
       'descripcion_del_proceso', 'tipo_de_contrato', 'modalidad_de_contratacion',
        'fecha_de_firma',  'fecha_de_inicio_de_ejecucion', 'fecha_de_fin_de_ejecucion',
        'tipodocproveedor','documento_proveedor', 'proveedor_adjudicado', 'valor_del_contrato',
        'origen_de_los_recursos', 'dias_adicionados','SECOPII']]
    SecopI_list=results.values.tolist()
    start = start + chunk_size
    
    ##Insert into database
    try:
        ##Connection
        conn = psycopg2.connect(conn_string)
        cursor = conn.cursor()
        cursor.executemany(insert_table_query,SecopI_list)
        conn.commit()
        print("inserting records ",start)
    except (Exception, psycopg2.DatabaseError) as error :
        print ("Error while inserting records in secop:", error)
    finally:
        #closing database connection.
            if(conn):
                cursor.close()
                conn.close()
                #print("PostgreSQL connection is closed") 
     # If we have fetched all of the records, bail out
    if (start > int(count_SECOPII[0]['COUNT']) ):
        break        

In [19]:
## Control of records inserted:
conn=psycopg2.connect(conn_string)
cursor=conn.cursor()
cursor.execute('select count(*)from secop;')
records = cursor.fetchall()
conn.commit()
cursor.close()
conn.close

if records != int(count_SECOPII[0]['COUNT']):
    print("There is a difference in the number of records from SECOPII (Datos Abiertos) with the target Postgres DB.")
else:
    print("The number of records inserted in the DB is equal to the number of records read from SECOPII (Datos Abiertos).")

There is a difference in the number of records from SECOPII (Datos Abiertos) with the target Postgres DB.


In [ ]:
## Inserting information of SECOP I
%%time
insert_table_query = '''INSERT INTO secop(
    nombre_entidad, nit_entidad, departamento, ciudad, 
    orden, id_contrato, estado_contrato, codigo_de_categoria_principal, 
    descripcion_del_proceso, tipo_de_contrato, modalidad_de_contratacion, 
    fecha_de_firma, fecha_de_inicio_de_ejecucion, fecha_de_fin_de_ejecucion, 
    tipodocproveedor, documento_proveedor, proveedor_adjudicado, 
    valor_del_contrato, origen_de_los_recursos, dias_adicionados, fuente)
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
start = 0            # Start at 0
chunk_size = 20000    # Fetch 2000 rows at a time
results =[]          # Empty out our result list
while True:
    
     # Fetch the set of records starting at 'start'
    results = client.get(secopI, offset=start, limit=chunk_size, select='''nombre_de_la_entidad, nit_de_la_entidad,
                                        departamento_entidad,municipio_entidad,nivel_entidad,
                                        numero_del_contrato,estado_del_proceso,
                                        id_objeto_a_contratar,detalle_del_objeto_a_contratar,
                                        tipo_de_contrato,regimen_de_contratacion,
                                        fecha_de_firma_del_contrato, fecha_ini_ejec_contrato,
                                        fecha_fin_ejec_contrato,tipo_identifi_del_contratista,
                                        identificacion_del_contratista, nom_raz_social_contratista,
                                        valor_contrato_con_adiciones,origen_de_los_recursos,
                                        tiempo_adiciones_en_dias,"SECOPI"''')

    results = pd.DataFrame(results)
    
    results = results[['nombre_de_la_entidad','nit_de_la_entidad', 'departamento_entidad', 'municipio_entidad',
                          'nivel_entidad', 'numero_del_contrato', 'estado_del_proceso', 'id_objeto_a_contratar',
                          'detalle_del_objeto_a_contratar', 'tipo_de_contrato', 'regimen_de_contratacion',
                          'fecha_de_firma_del_contrato', 'fecha_ini_ejec_contrato', 'fecha_fin_ejec_contrato',
                          'tipo_identifi_del_contratista', 'identificacion_del_contratista', 'nom_raz_social_contratista',
                          'valor_contrato_con_adiciones', 'origen_de_los_recursos', 'tiempo_adiciones_en_dias',
                          'SECOPI']]
    
    SecopI_list=results.values.tolist()
    start = start + chunk_size
    
    ##Insert into database
    try:
        ##Connection
        conn = psycopg2.connect(conn_string)
        cursor = conn.cursor()
        cursor.executemany(insert_table_query,SecopI_list)
        conn.commit()
        %time
        print("inserting records ",start)
    except (Exception, psycopg2.DatabaseError) as error :
        print ("Error while inserting records in secop:", error)
    finally:
        #closing database connection.
            if(conn):
                cursor.close()
                #conn.close()
                #print("PostgreSQL connection is closed")
                
    ####    
     # If we have fetched all of the records, bail out
    if (start > int(count_SECOPI[0]['COUNT']) ):
        break   

In [20]:
pwd

'/home/ec2-user'